In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Oasis 정형 데이터 분석

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/yonghyeun/Dementia_Analysis/main/data/oasis/oasis%20demographic.csv')

df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [5]:
df['CDR'].value_counts()

0.0    206
0.5    123
1.0     41
2.0      3
Name: CDR, dtype: int64

In [6]:
cond = (df['Group'] == 'Converted')

df[cond]['CDR'].value_counts() # converted 군의 CDR 이 0.0 인 경우엔 non-demented 로서의 첫 방문으로 가정 
                            # 0.5 가 되었고 visit 이 최소인 경우엔 demented 로서의 첫 방문으로 가정하자

0.5    19
0.0    18
Name: CDR, dtype: int64

In [7]:
cond = (df['Group'] == 'Converted') & (df['CDR'] == 0.0)

df.loc[cond, 'Group'] = 'Nondemented' # converted 인데 CDR 이 0.0 인 경우엔 Non demented 로 변경

In [8]:
cond = (df['Group'] == 'Converted') & (df['CDR'] == 0.5) 

df.loc[cond, 'Group'] = 'Mild demented' # converted CDR 이 0.5 인 경우엔 mild demented 로 변경

In [9]:
cond = (df['Group'] == 'Demented') & (df['CDR'] == 0.5) 

df.loc[cond, 'Group'] = 'Mild demented'

In [10]:
cond = (df['Group'] == 'Demented') & (df['CDR'] >= 1) 

df.loc[cond, 'Group'] = 'Moderate demented'

In [18]:
cond = (df['Group'] == 'Nondemented') & (df['CDR'] > 0) # CDR 이 0.5 이상이면서 Nondemented 인 경우엔 이상치러 초리하자

df = df.drop(index = df[cond].index)

In [22]:
mri_label = df[['Subject ID','MRI ID','Group']]
mri_label.to_csv('MRI_LABEL.csv',index = False) # MRI 용 라벨 

mri_label.head()

,Subject ID,MRI ID,Group
0,OAS2_0001,OAS2_0001_MR1,Nondemented
1,OAS2_0001,OAS2_0001_MR2,Nondemented
2,OAS2_0002,OAS2_0002_MR1,Mild demented
3,OAS2_0002,OAS2_0002_MR2,Mild demented
4,OAS2_0002,OAS2_0002_MR3,Mild demented


In [23]:
id = df['Subject ID'].unique()
S
df_list = []

for _ in id:
    
    visits = df.loc[df['Subject ID'] == _,'Visit'].values
    
    min_visit = np.min(visits)
    
    cond = (df['Subject ID'] == _) & (df['Visit'] == min_visit)
    
    df_list.append(df.loc[cond])

In [24]:
df = pd.concat(df_list)

df

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
2,OAS2_0002,OAS2_0002_MR1,Mild demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
5,OAS2_0004,OAS2_0004_MR1,Nondemented,1,0,F,R,88,18,3.0,28.0,0.0,1215,0.710,1.444
7,OAS2_0005,OAS2_0005_MR1,Nondemented,1,0,M,R,80,12,4.0,28.0,0.0,1689,0.712,1.039
10,OAS2_0007,OAS2_0007_MR1,Mild demented,1,0,M,R,71,16,NaN,28.0,0.5,1357,0.748,1.293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,OAS2_0182,OAS2_0182_MR1,Mild demented,1,0,M,R,73,12,NaN,23.0,0.5,1661,0.698,1.056
361,OAS2_0183,OAS2_0183_MR1,Nondemented,1,0,F,R,66,13,2.0,30.0,0.0,1495,0.746,1.174
365,OAS2_0184,OAS2_0184_MR1,Mild demented,1,0,F,R,72,16,3.0,24.0,0.5,1354,0.733,1.296
367,OAS2_0185,OAS2_0185_MR1,Mild demented,1,0,M,R,80,16,1.0,28.0,0.5,1704,0.711,1.030
